In [1]:
import jupyter_black
from IPython.display import display


jupyter_black.load(line_length=999)

In [2]:
import os
import pandas as pd
import torch
import torchvision
from tqdm.notebook import tqdm

In [3]:
class Person:
    features_type1 = [
        "ACCELERATION",
        "GYROSCOPE",
        "LIGHT",
        "LOCATION",
        "ORIENTATION",
        "PROXIMITY",
        "STEPS",
    ]

    features_type2 = [
        "BLUETOOTH",
        "CALL_RECORDS",
    ]

    def __init__(self, sid: str):
        self.sid = sid
        self.path = os.path.join(os.getcwd(), "dataset", sid)
        self.date_folders = os.listdir(self.path)

    def load(self, date_folder: str, feature: str):
        file_path = os.path.join(self.path, date_folder, f"{feature}.txt")
        if os.path.exists(file_path):
            if feature in self.features_type1:
                return self._read_type1(file_path)
            elif feature in self.features_type2:
                return self._read_type2(file_path)
        else:
            return pd.DataFrame()

    @staticmethod
    def _read_type1(file_path: str):
        with open(file_path, "r") as file:
            lines = file.readlines()

        check = len(lines[0].strip().split("|"))
        columns = ["Timestamp"] + [f"Column{i}" for i in range(1, check)]
        data = []
        for line in lines:
            row = line.strip().split("|")
            if len(row) == check:
                data.append(row)

        df = pd.DataFrame(data, columns=columns)
        for column in columns[1:]:
            try:
                df[column] = df[column].astype(float)
            except ValueError:
                pass
        return df

    @staticmethod
    def _read_type2(file_path: str):
        return 0

    def dump(self, dir_path: str, bar: bool = False):
        os.makedirs(dir_path, exist_ok=True)

        date_folders = tqdm(self.date_folders) if bar else self.date_folders

        for date_folder in date_folders:
            # for feature in self.features_type1 + self.features_type2:
            for feature in self.features_type1:
                df = self.load(date_folder, feature)
                if not df.empty:
                    df.insert(0, "SID", self.sid)
                    feature_dir = os.path.join(dir_path, feature)
                    os.makedirs(feature_dir, exist_ok=True)
                    output_path = os.path.join(feature_dir, f"{date_folder}.csv")
                    df.to_csv(output_path, index=False)

In [4]:
person = Person("mzy")

# for date_folder in tqdm(person.date_folders):
#     for feature in tqdm(person.features_type1):
#         df = person.load(date_folder, feature)

person.dump("dataset_csv", bar=True)

  0%|          | 0/35 [00:00<?, ?it/s]